# La reconnaissance des chiffres manuscrits MNIST (TensorFlow)



### Description du données Mnist

L'ensemble de données MNIST se compose d'un total de 70 000 images ; 60 000 images de formation (utilisés pour créer un modèle IR) et 10 000 images de test (utilisés pour évaluer l'exactitude du modèle). Chaque image MNIST est une image numérisée d'un personnage à un seul chiffre manuscrites. Chaque image est 28 x 28 pixels de taille. Chaque valeur de pixel est entre 0, qui représente le blanc, et 255, ce qui représente le noir. Valeurs des pixels intermédiaires représentent des nuances de gris.

![title](MNIST.png)


# Chargement du jeu de donnée d'Images Mnist

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets('yann.lecun.com/exdb/mnist/', one_hot=True)

C:\Users\Chhimy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Extracting yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# La Configuration des hyperparamètres

In [2]:
hidden_layer_size=1000 # la taille de chaque couche caché est de 1000 neurones
input_size=784 # chaque Image est de 28*28=784 pixels c-à-d nous avons 784 entrés
output_size=10 # la sortie est une classe de 10 chiffres de 0 jusqu'à 9

# La fonction tf.reset_default_graph() permet d'effacer les anciens paramètres. 
# A partir de là, une formation complètement nouvelle commence.
tf.reset_default_graph()

# Déclarer les caractères génériques dans lesquels les données seront introduites.
inputs=tf.placeholder(tf.float32,[None,input_size])
targets=tf.placeholder(tf.float32,[None,output_size])

# Introduction du modèle

nous allons utiliser trois des couches cachées(hidden layers) pour trainer notre modèle, en effet on a choisi 2 couches mais la précision n'était pas bonne (95%) c'est pour on a augmenté le nombre des couches pour augmenter la précision du modèle.

![title](réseaux.png)

In [3]:
# Poids et biais pour la première combinaison linéaire entre les entrées et la première couche cachée.
# la méthode get_variable pour utiliser l'initialisateur par défaut de TensorFlow qui est Xavier.
weights_1=tf.get_variable("weights_1",[input_size,hidden_layer_size])
biases_1=tf.get_variable("biases_1",[hidden_layer_size])

# Fonctionnement entre les entrées et la première couche cachée.
# Nous avons choisi ReLu comme fonction d'activation. Vous pouvez essayer de jouer avec différentes non-linéarités.
outputs_1=tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1)


# Poids et biais pour la deuxième combinaison linéaire.
# C'est entre la première et la deuxième couche cachée.
weights_2=tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2=tf.get_variable("biases_2",[hidden_layer_size])

# Opération entre la première et la deuxième couche cachée. Encore une fois, nous utilisons ReLu.
outputs_2=tf.nn.relu(tf.matmul(outputs_1,weights_2)+biases_2)


# N'oubliez pas de changer la forme des poids_3 et des biais_3. 
# Ils menaient à la couche de sortie, mais maintenant ils mènent à la troisième couche cachée.
weights_3=tf.get_variable("weights_3",[hidden_layer_size,hidden_layer_size])
biases_3=tf.get_variable("biases_3",[hidden_layer_size])

# Créer une variable output_3. Je vais utiliser ReLu encore une fois.
outputs_3=tf.nn.sigmoid(tf.matmul(outputs_2,weights_3)+biases_3)


# les poids_4 et les biais_4 sont liés directement à les sorties(outputs) de note modèle.
weights_4=tf.get_variable("weights_4",[hidden_layer_size,output_size])
biases_4=tf.get_variable("biases_4",[output_size])

# Les sorties(outputs) sont fonction des sorties_3, des poids_4 et des biais_4.
outputs = tf.matmul(outputs_3, weights_4) + biases_4



### the loss function (la fonction de perte)

In [4]:
# Calculer la fonction de perte pour chaque paire sortie(output) / cible(target).
# La fonction utilisée est la même que l'application de softmax à la dernière couche et le calcul de l'entropie croisée.
# avec la fonction que nous avons vu dans les conférences. Cette fonction, cependant, les combine d'une manière intelligente, 
# ce qui le rend à la fois plus rapide et plus stable numériquement (lorsqu'il s'agit de très petits nombres).
# Logits signifie ici : probabilités non mises à l'échelle (donc, les sorties, avant qu'elles ne soient mises à l'échelle par le softmax)
# les étiquettes sont les cibles.
loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)

# Obtenir la moyen de perte.
mean_loss=tf.reduce_mean(loss)

# Définir l'étape d'optimisation. Utilisation d'optimiseurs adaptatifs comme Adam dans TensorFlow.
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss) 



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### La mésure de prècision

la précision est le facteur qui détermine la puissance de notre modèle puisque nous allons la calculer sur le jeu de données d'entrainement (Train) et tester sa performance sur le jeu de données de test(Test).

In [5]:
# Obtenir un 0 ou un 1 pour chaque entrée du lot en indiquant si la réponse correcte est sortie sur les 10.
out_equals_target=tf.equal(tf.argmax(outputs,1),tf.argmax(targets,1))

# Obtenir la précision moyenne des sorties.
accuracy=tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

### lancement d'une session

pour démarrer le programme.

In [6]:
# Déclarer la variable de session.
sess=tf.InteractiveSession()

# Initialiser les variables. L'initialisateur par défaut est Xavier.
initializer=tf.global_variables_initializer()
sess.run(initializer)

# l'entrainement du modèle (Train)

In [7]:
# la mise en lot de jeu de données d'entrainement(train).
batch_size=200

# Calculez le nombre de lots par époque pour l'ensemble d'entraînement {(60000/100)=60 batches} .
batches_number= mnist.train._num_examples // batch_size

# Définir un nombre minimum d'époques à pourcourir avant d'arréter le traitement.
max_epochs=50

# Gardez la trace de la perte de validation de l'époque précédente.
# Si la perte de validation augmente, nous voulons déclencher un arrêt anticipé.
# Nous l'avons d'abord fixé à un nombre arbitrairement élevé pour nous assurer de ne pas le déclencher.
# à la première époque.
prev_validation_loss= 9999999.

# importer la classe time pour déterminer le temps d'exécution de l'algorithme
import time
start_time=time.time()


# Créez une boucle pour les époques. Epoch_counter est une variable qui part automatiquement de 0.
for epoch_counter in range(max_epochs):
    
    
    # Gardez une trace de la somme des pertes de lots à l'époque.
    curr_epoch_loss= 0.
    
    
     # Itérer sur les lots de cette époque.
    for batch_counter in range(batches_number):
        
        
        # Le lot d'entrée et le lot cible sont des valeurs assignées à partir de l'ensemble de données du train.
        input_batch,target_batch=mnist.train.next_batch(batch_size)
         
            
        # Exécutez l'étape d'optimisation et obtenez la perte moyenne pour ce lot.
        # On l'alimente avec les entrées et les cibles qu'on vient d'avoir dans les données du train.
        _, batch_loss=sess.run([optimizer,mean_loss],
                                  feed_dict={inputs:input_batch, targets:target_batch})    
        
        
        
        # incrémenter la somme des pertes de lots.
        curr_epoch_loss+=batch_loss
    
    
    # Jusqu'à présent curr_epoch_loss contenait la somme de tous les lots de l'époque
    # Nous voulons trouver la moyenne des pertes de lots sur l'ensemble de l'époque
    # La perte moyenne par lot est une bonne approximation de la perte de l'époque actuelle.
    curr_epoch_loss /= batches_number
    
    
    # Obtenir le lot d'entrée et le lot cible à partir de l'ensemble de données de validation.
    input_batch,target_batch=mnist.validation.next_batch(mnist.validation._num_examples)

    
    # Exécuter sans l'étape d'optimisation simplement propager vers l'avant (forward propagation).
    # A la fin de chaque époque, obtenez la perte de validation et la précision.
    validation_loss,validation_accuracy=sess.run([mean_loss,accuracy], 
                                                 feed_dict= {inputs:  input_batch,
                                                             targets: target_batch})
    # Imprimer les statistiques pour l'époque actuelle
    # Compteur d'époque + 1, car epoch_counter démarre automatiquement à partir de 0, au lieu de 1
    # Nous formaterons les pertes avec 3 chiffres après le point
    # Nous formaterons l'exactitude en pourcentages pour faciliter l'interprétation.
    print('Epoch'+str(epoch_counter+1)+
         '.Training loss'+'{0:.3f}'.format(curr_epoch_loss)+
         '.Validation loss'+'{0:.3f}'.format(validation_loss)+
         '.Validation accuracy'+'{0:.2f}'.format(validation_accuracy*100.)+'%')
     
        
    # Déclenchement de l'arrêt anticipé si la perte de validation commence à augmenter.
    if validation_loss > prev_validation_loss:
        break
    
    # Stockez la perte de validation de cette époque pour l'utiliser comme perte de validation précédente lors de la prochaine itération.
    prev_validation_loss = validation_loss    

# Pas essentiel, mais c'est bien de savoir quand l'algorithme a cessé de fonctionner dans la section output,
# et savoir le nombre d'époque qu'il a traversé.
print('end of trainig')     

# Le temps de l'entrainement.
print("Training time: %s seconds" % (time.time() - start_time))

Epoch1.Training loss0.258.Validation loss0.096.Validation accuracy97.06%
Epoch2.Training loss0.084.Validation loss0.081.Validation accuracy97.76%
Epoch3.Training loss0.051.Validation loss0.069.Validation accuracy97.98%
Epoch4.Training loss0.036.Validation loss0.059.Validation accuracy98.22%
Epoch5.Training loss0.028.Validation loss0.071.Validation accuracy97.88%
end of trainig
Training time: 281.324090719223 seconds


# Le test de la précision prédictive

Comme nous en avons discuté dans les exposés, après un apprentissage sur les ensembles de d'entrainement et de validation, nous testons la puissance de prédiction finale de notre modèle en l'exécutant sur l'ensemble de données de test que l'algorithme n'a jamais vu auparavant.

Il est très important de se rendre compte que le fait de manipuler les hyperparamètres surdimensionne l'ensemble des données de validation. Le test est l'instance finale absolue. Vous ne devez pas tester avant d'avoir terminé d'ajuster votre modèle.



In [8]:
# Obtenir le lot d'entrée et le lot cible à partir de l'ensemble de données de test.
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy], 
                         feed_dict={inputs: input_batch,
                                    targets: target_batch})


# La précision du test est une liste avec 1 valeur, nous voulons donc en extraire la valeur, en utilisant x[0].
test_accuracy_percent=test_accuracy[0]*100.


# retourné la précision du test en pourcentages
print('Test accuracy:'+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy:98.09%


En utilisant le modèle initial et les hyperparamètres donnés dans ce projet, la précision de l'essai final devrait se situer approximativement entre 97 % et 98 %. Chaque fois que le code est réexécuté, nous obtenons une précision différente car les lots sont mélangés, les poids sont initialisés d'une manière différente, etc.

Enfin, nous avons délibérément atteint une solution sous-optimale, de sorte que vous pouvez avoir de l'espace pour construire dessus.
